In [18]:
#encoding=utf-8
import pandas as pd

loans = pd.read_csv('loans.csv')
del loans['id']

In [19]:
loans

,age,job,house,credit,class
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [20]:
def change(y):
    x = y.decode('utf-8')
    if x == u'青年':
        x = 0
    elif x == u'中年':
        x = 1
    elif x == u'老年':
        x = 2
    elif x == u'否':
        x = 0
    elif x == u'是':
        x = 1
    elif x == u'一般':
        x = 0
    elif x == u'好':
        x = 1
    elif x == u'非常好':
        x = 2
    return x

loans = loans.applymap(change)

In [21]:
loans

,age,job,house,credit,class
0,0,0,0,0,0
1,0,0,0,1,0
2,0,1,0,1,1
3,0,1,1,0,1
4,0,0,0,0,0
5,1,0,0,0,0
6,1,0,0,1,0
7,1,1,1,1,1
8,1,0,1,2,1
9,1,0,1,2,1


In [22]:
loans.age.value_counts()

2    5
1    5
0    5
Name: age, dtype: int64

In [24]:
total_count  = len(loans);total_count




15